# Cafe Sales Data Cleaning

In this notebook, I clean the dirty cafe sales dataset. The data has many problems like missing values and invalid entries (ERROR, UNKNOWN). I will fix these issues and prepare the data for analysis.

In [4]:
import pandas as pd 

## Load the Data
First, I import pandas and load the CSV file. I use `info()` and `head()` to understand the data structure and see what problems exist.

In [5]:
data=pd.read_csv('dirty_cafe_sales.csv')
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Transaction ID    10000 non-null  str  
 1   Item              9667 non-null   str  
 2   Quantity          9862 non-null   str  
 3   Price Per Unit    9821 non-null   str  
 4   Total Spent       9827 non-null   str  
 5   Payment Method    7421 non-null   str  
 6   Location          6735 non-null   str  
 7   Transaction Date  9841 non-null   str  
dtypes: str(8)
memory usage: 625.1 KB


In [6]:
data.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [7]:
data.isna().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

## Rename Columns
I renamed the columns to remove spaces. This makes them easier to work with in code. For example, I can write `data.Price_per_unit` instead of `data['Price Per Unit']`.

In [8]:
data.rename(columns={
    "Transaction ID": "Transaction_ID",
    "Price Per Unit": "Price_per_unit",
    "Total Spent": "Total_spent",
    "Payment Method": "Payment_method",
    "Transaction Date": "Transaction_date"
}, inplace=True)

## Remove Duplicates
I dropped duplicate rows because I don't want to count the same transaction twice. This would give wrong results in analysis.

In [9]:
data.drop_duplicates(inplace=True)


## Explore Item and Price Data
Before cleaning, I need to understand the data. I checked what items are sold and their prices. I also looked for ERROR and UNKNOWN values. This helps me decide how to fix the bad values later.

In [10]:
data['Price_per_unit'].value_counts()


Price_per_unit
3.0        2429
4.0        2331
2.0        1227
5.0        1204
1.0        1143
1.5        1133
ERROR       190
UNKNOWN     164
Name: count, dtype: int64

In [11]:
data['Item'].value_counts()


Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64

In [12]:
tmp = data[['Item', 'Price_per_unit']].dropna()
tmp['Price_per_unit'] = pd.to_numeric(tmp['Price_per_unit'], errors='coerce')
tmp = tmp.dropna()

price_per_item = (
    tmp.groupby('Item')['Price_per_unit']
    .apply(lambda s: sorted(s.unique()))
    .reset_index(name='prices')
)
price_per_item.head(30)

,Item,prices
0,Cake,[3.0]
1,Coffee,[2.0]
2,Cookie,[1.0]
3,ERROR,"[1.0, 1.5, 2.0, 3.0, 4.0, 5.0]"
4,Juice,[3.0]
5,Salad,[5.0]
6,Sandwich,[4.0]
7,Smoothie,[4.0]
8,Tea,[1.5]
9,UNKNOWN,"[1.0, 1.5, 2.0, 3.0, 4.0, 5.0]"


In [13]:
data.loc[data['Item'] == 'Salad']

,Transaction_ID,Item,Quantity,Price_per_unit,Total_spent,Payment_method,Location,Transaction_date
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
10,TXN_2548360,Salad,5,5.0,25.0,Cash,Takeaway,2023-11-07
15,TXN_2847255,Salad,3,5.0,15.0,Credit Card,In-store,2023-11-15
37,TXN_1080432,Salad,2,5.0,10.0,Credit Card,In-store,2023-04-29
45,TXN_5455792,Salad,3,5.0,15.0,Cash,NaN,2023-03-22
...,...,...,...,...,...,...,...,...
9928,TXN_5171519,Salad,3,5.0,15.0,NaN,In-store,2023-04-06
9940,TXN_8273780,Salad,2,5.0,10.0,Digital Wallet,Takeaway,2023-10-15
9942,TXN_5344848,Salad,1,5.0,5.0,Digital Wallet,Takeaway,2023-09-27
9971,TXN_6120851,Salad,5,5.0,25.0,Cash,Takeaway,2023-02-04


In [14]:
data['Payment_method'].value_counts()


Payment_method
Digital Wallet    2291
Credit Card       2273
Cash              2258
ERROR              306
UNKNOWN            293
Name: count, dtype: int64

## Clean Payment Method
I checked the Payment_method column and found ERROR and UNKNOWN values. These are not real payment methods, so I need to fix them.

In [ ]:

data['Payment_method'] = data['Payment_method'].replace(['ERROR','UNKNOWN'], None) 
data['Payment_method'].value_counts()


Payment_method
Digital Wallet    2291
Credit Card       2273
Cash              2258
Name: count, dtype: int64

I replaced ERROR and UNKNOWN with `None`, then filled missing values with "Unknown". 

**Why "Unknown"?** I don't want to lose the transaction data (the sale still happened), but I also can't guess what payment method was used. So "Unknown" is honest - it says "I don't know".

In [16]:
data["Payment_method"] = data["Payment_method"].fillna("Unknown")

In [17]:
data['Payment_method'].value_counts()


Payment_method
Unknown           3178
Digital Wallet    2291
Credit Card       2273
Cash              2258
Name: count, dtype: int64

In [18]:
data['Location'].value_counts()

Location
Takeaway    3022
In-store    3017
ERROR        358
UNKNOWN      338
Name: count, dtype: int64

## Clean Location
I checked the Location column and found the same problem - ERROR and UNKNOWN values. I will use the same approach as Payment Method.

In [19]:
pd.crosstab(data["Location"], data["Payment_method"])

Payment_method,Cash,Credit Card,Digital Wallet,Unknown
Location,,,,
ERROR,87,82,78,111
In-store,702,681,695,939
Takeaway,664,669,744,945
UNKNOWN,74,84,71,109


In [20]:
data['Location'] = data['Location'].replace(['ERROR','UNKNOWN'], None) 
data["Location"] = data["Location"].fillna("Unknown")

I replaced ERROR and UNKNOWN with `None`, then filled with "Unknown".

**Why keep these rows?** The location might be unknown, but the other data (item, price, quantity) is still useful for analysis. I don't want to throw away good data just because one column is missing.

In [21]:
data['Location'].value_counts()

Location
Unknown     3961
Takeaway    3022
In-store    3017
Name: count, dtype: int64

In [22]:
data.head()

,Transaction_ID,Item,Quantity,Price_per_unit,Total_spent,Payment_method,Location,Transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [23]:
pd.crosstab(data["Location"], data["Payment_method"])

Payment_method,Cash,Credit Card,Digital Wallet,Unknown
Location,,,,
In-store,702,681,695,939
Takeaway,664,669,744,945
Unknown,892,923,852,1294


## Exploring Numeric Columns
Now I look at the numeric columns: Price_per_unit and Item. I use crosstab to see the relationship between items and their prices. This shows me which price belongs to which item.

In [24]:
data.head()

,Transaction_ID,Item,Quantity,Price_per_unit,Total_spent,Payment_method,Location,Transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [25]:
pd.crosstab(data["Item"], data["Price_per_unit"])

Price_per_unit,1.0,1.5,2.0,3.0,4.0,5.0,ERROR,UNKNOWN
Item,,,,,,,,
Cake,0,0,0,1085,0,0,19,14
Coffee,0,0,1108,0,0,0,18,20
Cookie,1026,0,0,0,0,0,21,21
ERROR,34,37,31,77,61,39,3,3
Juice,0,0,0,1110,0,0,26,18
Salad,0,0,0,0,0,1082,34,16
Sandwich,0,0,0,0,1082,0,13,19
Smoothie,0,0,0,0,1036,0,19,17
Tea,0,1023,0,0,0,0,25,21


In [26]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction_ID    10000 non-null  str   
 1   Item              9667 non-null   str   
 2   Quantity          9862 non-null   str   
 3   Price_per_unit    9821 non-null   str   
 4   Total_spent       9827 non-null   str   
 5   Payment_method    10000 non-null  object
 6   Location          10000 non-null  object
 7   Transaction_date  9841 non-null   str   
dtypes: object(2), str(6)
memory usage: 625.1+ KB


## Fix Price Per Unit
I created a mapping of each item to its correct price (e.g., Coffee = $2, Sandwich = $4). When Price_per_unit is ERROR or UNKNOWN, I replace it with the correct price based on the Item name.

**Why this works:** Each item has a fixed price in this cafe. If I know the item, I know the price.

I also dropped rows where I couldn't determine the price (when both Item and Price are bad).

In [ ]:
item_price_map = {
    'Cake': '3',
    'Coffee': '2',
    'Cookie': '1',
    'Juice': '3',
    'Salad': '5',
    'Sandwich': '4',
    'Smoothie': '4',
    'Tea': '1.5',
}

mask_bad = data['Price_per_unit'].isin(['ERROR', 'UNKNOWN'])
data.loc[mask_bad, 'Price_per_unit'] = data.loc[mask_bad, 'Item'].map(item_price_map)
data['Price_per_unit'] = data['Price_per_unit'].fillna(data['Item'].map(item_price_map))
data.dropna(subset=[ 'Price_per_unit'], inplace=True)
data['Price_per_unit'] = pd.to_numeric(data['Price_per_unit'], errors='coerce')

data['Price_per_unit'].value_counts(dropna=False).head(20)

Price_per_unit
3.0    2544
4.0    2440
2.0    1284
5.0    1270
1.0    1209
1.5    1199
Name: count, dtype: int64

In [28]:
pd.crosstab(data["Item"], data["Price_per_unit"],dropna=False)

Price_per_unit,1.0,1.5,2.0,3.0,4.0,5.0
Item,,,,,,
Cake,0,0,0,1139,0,0
Coffee,0,0,1165,0,0,0
Cookie,1092,0,0,0,0,0
ERROR,34,37,31,77,61,39
Juice,0,0,0,1171,0,0
Salad,0,0,0,0,0,1148
Sandwich,0,0,0,0,1131,0
Smoothie,0,0,0,0,1096,0
Tea,0,1089,0,0,0,0


## Fix Item Names
When Item is UNKNOWN or ERROR, I guess it from the price. For example, if price is $2, it must be Coffee.

**Problem:** Some prices have multiple items. $3 could be Cake or Juice. $4 could be Sandwich or Smoothie.

**My decision:** I picked the most popular item for each price:
- $3 → Juice (more popular than Cake)"The Mood"
- $4 → Sandwich (more popular than Smoothie)"The Mood"

This isn't perfect, but it's better than losing the data.

In [29]:
# Fix Item عندما يكون UNKNOWN/ERROR اعتماداً على السعر
mask2 = data['Item'].isin(['UNKNOWN', 'ERROR',None])

# حول السعر لرقمي أولاً (لو كان نص)
price_num = pd.to_numeric(data.loc[mask2, 'Price_per_unit'], errors='coerce')

# السعر -> Item (فقط الأسعار اللي لها Item واحد واضح)
price_to_item = {
    1.0: 'Cookie',
    1.5: 'Tea',
    2.0: 'Coffee',
    5.0: 'Salad',
    3.0: 'Juice',# اخترتها لانها الاكثر طلبا
    4.0: 'Sandwich' #اخترتها لانها الاكثر طلبا
    
}

data.loc[mask2, 'Item'] = price_num.map(price_to_item)

data['Item'].value_counts().head(20)

Item
Juice       1405
Sandwich    1344
Coffee      1284
Salad       1270
Cookie      1209
Tea         1199
Cake        1139
Smoothie    1096
Name: count, dtype: int64

In [30]:
pd.crosstab(data["Item"], data["Price_per_unit"],dropna=False)

Price_per_unit,1.0,1.5,2.0,3.0,4.0,5.0
Item,,,,,,
Cake,0,0,0,1139,0,0
Coffee,0,0,1284,0,0,0
Cookie,1209,0,0,0,0,0
Juice,0,0,0,1405,0,0
Salad,0,0,0,0,0,1270
Sandwich,0,0,0,0,1344,0
Smoothie,0,0,0,0,1096,0
Tea,0,1199,0,0,0,0


In [31]:
data.info()

<class 'pandas.DataFrame'>
Index: 9946 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction_ID    9946 non-null   str    
 1   Item              9946 non-null   str    
 2   Quantity          9808 non-null   str    
 3   Price_per_unit    9946 non-null   float64
 4   Total_spent       9774 non-null   str    
 5   Payment_method    9946 non-null   object 
 6   Location          9946 non-null   object 
 7   Transaction_date  9789 non-null   str    
dtypes: float64(1), object(2), str(5)
memory usage: 699.3+ KB


## Cleaning Total Spent & Quantity
These two columns are connected by a formula: **Total Spent = Quantity × Price Per Unit**

This means if one is missing, I can calculate it from the other two:
- Missing Quantity? → I calculate: Quantity = Total_spent / Price_per_unit
- Missing Total_spent? → I calculate: Total_spent = Quantity × Price_per_unit
- Both missing? → I have to drop the row (no way to recover)

In [32]:
data.head()

,Transaction_ID,Item,Quantity,Price_per_unit,Total_spent,Payment_method,Location,Transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [33]:
pd.crosstab(data['Total_spent'],data['Quantity'])

Quantity,1,2,3,4,5,ERROR,UNKNOWN
Total_spent,,,,,,,
1.0,215,0,0,0,0,7,7
1.5,187,0,0,0,0,4,5
10.0,0,245,0,0,254,7,7
12.0,0,0,439,449,0,16,13
15.0,0,0,208,0,486,9,14
16.0,0,0,0,425,0,7,6
2.0,227,247,0,0,0,7,9
20.0,0,0,0,227,480,16,13
25.0,0,0,0,0,246,6,4


### Fix Missing Quantity
I calculated the missing Quantity using: **Quantity = Total Spent / Price Per Unit**

For example: If Total_spent = $6 and Price_per_unit = $2, then Quantity = 6/2 = 3 items.

In [34]:
data['Quantity'].value_counts(dropna=False)

Quantity
5          2001
2          1966
4          1848
3          1839
1          1816
ERROR       169
UNKNOWN     169
NaN         138
Name: count, dtype: int64

In [35]:
# Fix Quantity عندما يكون UNKNOWN/ERROR/NaN باستخدام: Quantity = Total_spent / Price_per_unit
mask3 = data['Quantity'].isin(['UNKNOWN', 'ERROR']) | data['Quantity'].isna()

# تأكد أن الأعمدة رقمية
data['Total_spent'] = pd.to_numeric(data['Total_spent'], errors='coerce')
data['Price_per_unit'] = pd.to_numeric(data['Price_per_unit'], errors='coerce')
data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')

# احسب الكمية فقط للصفوف الناقصة
new_qty = data.loc[mask3, 'Total_spent'] / data.loc[mask3, 'Price_per_unit']
data.loc[mask3, 'Quantity'] = new_qty

# اختياري: تحويل Quantity لرقمي
# data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')

data['Quantity'].value_counts(dropna=False).head(20)

Quantity
5.0    2096
2.0    2048
3.0    1936
4.0    1926
1.0    1920
NaN      20
Name: count, dtype: int64

In [36]:
pd.crosstab(data['Total_spent'],data['Quantity'],dropna=False)

Quantity,1.0,2.0,3.0,4.0,5.0,NaN
Total_spent,,,,,,
1.0,232,0,0,0,0,0
1.5,205,0,0,0,0,0
2.0,242,253,0,0,0,0
3.0,467,262,196,0,0,0
4.0,436,242,0,242,0,0
4.5,0,0,225,0,0,0
5.0,244,0,0,0,222,0
6.0,0,511,242,217,0,0
7.5,0,0,0,0,237,0


### Fix Missing Total Spent
I calculated the missing Total_spent using: **Total Spent = Quantity × Price Per Unit**

For example: If Quantity = 3 and Price_per_unit = $2, then Total_spent = 3 × 2 = $6.

In [ ]:

mask3 = data['Total_spent'].isin(['UNKNOWN', 'ERROR']) | data['Total_spent'].isna()

new_qty = data.loc[mask3, 'Quantity'] * data.loc[mask3, 'Price_per_unit']
data.loc[mask3, 'Total_spent'] = new_qty


data['Total_spent'].value_counts(dropna=False).head(20)

Total_spent
6.0     1010
12.0     992
4.0      966
3.0      963
20.0     788
15.0     757
8.0      716
10.0     539
2.0      516
9.0      507
5.0      494
16.0     461
25.0     268
7.5      250
1.0      249
4.5      238
1.5      212
NaN       20
Name: count, dtype: int64

In [38]:
pd.crosstab(data['Total_spent'],data['Quantity'],dropna=False)

Quantity,1.0,2.0,3.0,4.0,5.0,NaN
Total_spent,,,,,,
1.0,249,0,0,0,0,0
1.5,212,0,0,0,0,0
2.0,255,261,0,0,0,0
3.0,481,273,209,0,0,0
4.0,464,249,0,253,0,0
4.5,0,0,238,0,0,0
5.0,259,0,0,0,235,0
6.0,0,531,254,225,0,0
7.5,0,0,0,0,250,0


### Drop Remaining Missing Values
After calculating what I can, some rows still have missing Total_spent or Quantity. These rows don't have enough data to recover, so I dropped them.

**Why drop?** If I can't calculate the value and I can't guess it, keeping the row would give wrong analysis results.

In [39]:
data.dropna(subset=['Total_spent','Quantity'], inplace=True)
data.isna().sum()

Transaction_ID        0
Item                  0
Quantity              0
Price_per_unit        0
Total_spent           0
Payment_method        0
Location              0
Transaction_date    157
dtype: int64

In [40]:
data.info()

<class 'pandas.DataFrame'>
Index: 9926 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction_ID    9926 non-null   str    
 1   Item              9926 non-null   str    
 2   Quantity          9926 non-null   float64
 3   Price_per_unit    9926 non-null   float64
 4   Total_spent       9926 non-null   float64
 5   Payment_method    9926 non-null   object 
 6   Location          9926 non-null   object 
 7   Transaction_date  9769 non-null   str    
dtypes: float64(3), object(2), str(3)
memory usage: 697.9+ KB


## Clean Transaction Date
I replaced ERROR and UNKNOWN dates with `None`, then dropped rows with missing dates.

**Why drop instead of fill?** I can't guess what date a transaction happened. Unlike Payment Method where I used "Unknown", a missing date makes the transaction useless for time-based analysis.

Finally, I converted the dates to datetime type so I can do date calculations easily.

In [41]:

data['Transaction_date'].value_counts(dropna=False)

Transaction_date
UNKNOWN       158
NaN           157
ERROR         142
2023-06-16     40
2023-02-06     39
             ... 
2023-09-24     15
2023-07-30     15
2023-03-11     14
2023-02-17     14
2023-07-22     13
Name: count, Length: 368, dtype: int64

In [42]:
data['Transaction_date']=data['Transaction_date'].replace(['UNKNOWN','ERROR'],None)

In [43]:
data['Transaction_date'].value_counts(dropna=False)

Transaction_date
None          300
NaN           157
2023-06-16     40
2023-02-06     39
2023-03-13     39
             ... 
2023-09-24     15
2023-07-30     15
2023-03-11     14
2023-02-17     14
2023-07-22     13
Name: count, Length: 367, dtype: int64

In [44]:
data.dropna(subset=['Transaction_date'], inplace=True)

In [45]:
data['Transaction_date'].value_counts(dropna=False)

Transaction_date
2023-06-16    40
2023-02-06    39
2023-03-13    39
2023-07-21    39
2023-07-24    39
              ..
2023-09-24    15
2023-07-30    15
2023-03-11    14
2023-02-17    14
2023-07-22    13
Name: count, Length: 365, dtype: int64

In [46]:
data.info()

<class 'pandas.DataFrame'>
Index: 9469 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction_ID    9469 non-null   str    
 1   Item              9469 non-null   str    
 2   Quantity          9469 non-null   float64
 3   Price_per_unit    9469 non-null   float64
 4   Total_spent       9469 non-null   float64
 5   Payment_method    9469 non-null   object 
 6   Location          9469 non-null   object 
 7   Transaction_date  9469 non-null   object 
dtypes: float64(3), object(3), str(2)
memory usage: 665.8+ KB


In [47]:
data['Transaction_date'] = pd.to_datetime(data['Transaction_date'], errors='coerce')
data['Transaction_date'].value_counts(dropna=False).head(20)

Transaction_date
2023-06-16    40
2023-02-06    39
2023-03-13    39
2023-07-21    39
2023-07-24    39
2023-01-05    38
2023-09-21    38
2023-06-18    37
2023-10-22    37
2023-01-25    37
2023-11-07    36
2023-04-30    36
2023-08-07    36
2023-01-12    36
2023-06-30    36
2023-04-06    35
2023-10-08    35
2023-03-26    35
2023-09-06    35
2023-06-28    35
Name: count, dtype: int64

### Convert Data Types
I converted Payment_method and Location to string type. This keeps the data types consistent and prevents any issues later in analysis.

In [48]:
data['Payment_method'] = data['Payment_method'].astype(str)
data['Location']=data['Location'].astype(str)
data.info()

<class 'pandas.DataFrame'>
Index: 9469 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction_ID    9469 non-null   str           
 1   Item              9469 non-null   str           
 2   Quantity          9469 non-null   float64       
 3   Price_per_unit    9469 non-null   float64       
 4   Total_spent       9469 non-null   float64       
 5   Payment_method    9469 non-null   str           
 6   Location          9469 non-null   str           
 7   Transaction_date  9469 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(3), str(4)
memory usage: 665.8 KB


## Create Season Feature
I added a new column called "Season" based on the month of the transaction:
- **Winter**: December, January, February
- **Spring**: March, April, May
- **Summer**: June, July, August
- **Fall**: September, October, November

**Why?** This helps me analyze sales by season. For example, I can see if coffee sells more in winter or if smoothies sell more in summer.

In [49]:
masks = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
}

data['Season'] = data['Transaction_date'].dt.month.map(masks).fillna('Unknown')
data.head()

,Transaction_ID,Item,Quantity,Price_per_unit,Total_spent,Payment_method,Location,Transaction_date,Season
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08,Fall
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16,Spring
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19,Summer
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27,Spring
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11,Summer


In [50]:
data.info()

<class 'pandas.DataFrame'>
Index: 9469 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction_ID    9469 non-null   str           
 1   Item              9469 non-null   str           
 2   Quantity          9469 non-null   float64       
 3   Price_per_unit    9469 non-null   float64       
 4   Total_spent       9469 non-null   float64       
 5   Payment_method    9469 non-null   str           
 6   Location          9469 non-null   str           
 7   Transaction_date  9469 non-null   datetime64[us]
 8   Season            9469 non-null   str           
dtypes: datetime64[us](1), float64(3), str(5)
memory usage: 739.8 KB


## Save Cleaned Data
Finally, I exported the cleaned data to a new CSV file. The original dirty file is preserved, and now I have a clean version ready for analysis.

In [51]:
data.to_csv('cleaned_cafe_sales.csv', index=False)